# Imports

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *

done1
stop
stop
done1
stop
done
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'13052018'

# explore BD-FD table:

In [ ]:
readCountFolder='Metabolon2'

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_RCfolder%s_18032018.xlsx' %readCountFolder
final_BD_FD_converter_mergedOnYear_16032018=pd.read_excel(file1)
final_BD_FD_converter_mergedOnYear_16032018.head()

In [ ]:
# how many samples are genotek and how many are swabs?
final_BD_FD_converter_mergedOnYear_16032018['isGenotek'].value_counts(dropna=False)

In [ ]:
final_BD_FD_converter_mergedOnYear_16032018['BD'].nunique()

In [ ]:
#divide sample to Genotek and no Genotek
noGenotek=final_BD_FD_converter_mergedOnYear_16032018[final_BD_FD_converter_mergedOnYear_16032018['isGenotek']!=1]
Genotek=final_BD_FD_converter_mergedOnYear_16032018[final_BD_FD_converter_mergedOnYear_16032018['isGenotek']==1]

In [ ]:
noGenotek['BD'].nunique()

In [ ]:
cardio=final_BD_FD_converter_mergedOnYear_16032018[final_BD_FD_converter_mergedOnYear_16032018['BD_index']>949]

In [ ]:
cardio.count()

In [ ]:
#define pnp samples only:
pnp=final_BD_FD_converter_mergedOnYear_16032018[final_BD_FD_converter_mergedOnYear_16032018['BD_index']<951]

In [ ]:
pnp.count()

In [ ]:
# unique BDs in the PNP cohort:
pnp['BD'].nunique()

In [ ]:
#define pnp swab samples only. there are 1446 such samples
pnp_swabs=pnp[pnp['isGenotek']!=1]
print len(pnp_swabs)

In [ ]:
#with currently 869 unique BD matched
pnp_swabs['BD'].nunique()

In [ ]:
#pnp BD samples with matched FD that has postHGF info: (921 samples currently)
pnp_postHGF=pnp[pnp['PostHGF'].notnull()]
pnp_postHGF['BD'].nunique()

In [ ]:
# pnp BD swab samples with postHGF info-two ways to calculate - in both we get 636 samples
pnp_postHGF_swab1=pnp_postHGF[pnp_postHGF['isGenotek']!=1]
pnp_postHGF_swab2=pnp_swabs[pnp_swabs['PostHGF'].notnull()]
print pnp_postHGF_swab1['BD'].nunique()
print pnp_postHGF_swab2['BD'].nunique()


In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP434samples','rb') as fp:
    pnp434samples=pickle.load(fp)
pnp434samples[-10:]

In [ ]:
#431 samples in this cohort have matched FD
pnp[pnp['BD'].isin(pnp434samples)]['BD'].nunique()

In [ ]:
#there are 405 out of 434 samples in the PNP434 cohort that have at least one swab FD sample:
pnp_swabs_434=pnp_swabs[pnp_swabs['BD'].isin(pnp434samples)]
print pnp_swabs_434['BD'].nunique()

In [ ]:
pnp_postHGF_434=pnp_postHGF[pnp_postHGF['BD'].isin(pnp434samples)]
print pnp_postHGF_434['BD'].nunique()

In [ ]:
#only 270 out of these samples have read count info!
pnp_swabs_434[pnp_swabs_434['PostHGF'].notnull()]['BD'].nunique()

In [ ]:
#266 out of 270 samples have at least 9000000 read counts.
pnp_swabs_434[pnp_swabs_434['PostHGF']>9000000]['BD'].nunique()

In [ ]:
#266 out of 270 samples have at least 9000000 read counts.
pnp_swabs_434[pnp_swabs_434['PostHGF']>8000000]['BD'].nunique()

NEED TO UNDERSTAND WHY SO MANY SAMPLES DON'T HAVE READ COUNT INFO!

## explore how many samples in the original file don't have this data

In [ ]:

folder='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut'
filenames = [f for f in listdir(folder) if isfile(join(folder, f))]
filenames

In [ ]:
f1='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/ReadCountSpidDF.dat'
ReadCountSpidDF=pd.read_pickle(f1)
ReadCountSpidDF.head()

In [ ]:
ReadCountSpidDF=ReadCountSpidDF.reset_index()

In [ ]:
print len(ReadCountSpidDF)

In [ ]:
ReadCountSpidDF[ReadCountSpidDF['PostHGF'].isnull()]

In [ ]:
FDlist=list(ReadCountSpidDF['FD'])

In [ ]:
f1='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/MPASpid.dat'
MPASpid=pd.read_pickle(f1)
MPASpid.head()

In [ ]:
print len(MPASpid.columns)

In [ ]:
for column in MPASpid.columns.values:
    newColumn=column.split('_')[0]
#     print column
#     print newColumn
    MPASpid=MPASpid.rename(columns={column:newColumn})
MPASpid.head()

In [ ]:
# check if all FDs in MPA file exist in read count file:
a=MPASpid.columns.values
b=FDlist
print len(set(a).intersection(b))

all samples in MPA file appear in the read count file! and thus have read count info

# filtering FD-based data:

1.take data df, 
2. merge with BD-FD conversion table
3.Edit sample names
4.Filter on isGenotek=0
5.Filter on minimal number of reads
6.Filter on meet location
7.Filter same person
8. Filter outliers
9. filter out non-complete matches if there is at least one complete match for the BD
9. filter for specific BD samples


## function:

In [ ]:
'''
### THIS FUNCTION WAS COPIED TO Feature_phenotype_functions.py - UPDATE THERE!!!

this function takes an FD-based data table (can use also other identifiers such as SPID, connection)
merge it with the BD-FD conversion table, and then filter the data based on specific sample list, only
swab samples and/or minimal reads per sample.

the required input:
dataDF: FD-based table, with samples as rows and phenotypes as columns (for example - microbiome species 
relative abundances) ####make sure the column to merge on is not the index!! ###
dataDFname - a string, such as MPA_s
dataMergeOn- the name of the column in the dataDF table to merge on ###don't forget to reset_index if necessary
BDFDMergeOn- the name of the column in the BD_FD table to merge on
Sample list - None, or python list of BD numbers in the format: ['BD1','BD2'...]
SampleListName-None or the list name
filterGenotek - True/False. True will lead to filtering out of all samples with isGenotek==1. samples with
no indication at all won't be filtered out.
filterMinimalReads - None, or a number. recommended value is 9000000
folderToSave=None or full path to the required folder

'''


def filter_data_on_FDproperties(readCountFolder,dataDF,dataDFname,dataMergeOn,BDFDMergeOn,SampleList=None,
                                SampleListName=None,filterGenotek=False, filterMinimalReads=9000000,
                               folderToSave=None):
    
      if BD_FD is None:
        print 'using BD_FD converter based on AllSeqProjects folder, date 17042018'
        f1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_RCfolderAllSeqProjects_17042018'          
        BD_FD = pd.read_pickle(f1)
    print 'row number in BD_FD table is %s' % len(BD_FD)
    
    #merging:
    print 'merging dataDF with FD-BD table:'

    dataWithMeta=pd.merge(dataDF,BD_FD,how='left', left_on=dataMergeOn, right_on=BDFDMergeOn)
    print 'row number in dataDF before merging was %s' %len(dataDF)
    print 'row number after merging is %s' %len(dataWithMeta)
    
    ### if required - filter on specific BDs, if not-filter on samples that have any BD:
    if SampleList is None:
        print 'filtering out all FD samples without BDs'
        dataWithMeta=dataWithMeta[dataWithMeta['BD'].notnull()]
        print 'row number after filtering is %s' %len(dataWithMeta)
    else:
        print 'filtering out all FD that are not matched to any BD in the sample list' 
        dataWithMeta=dataWithMeta[dataWithMeta['BD'].isin(SampleList)]
        print 'row number after filtering is %s' %len(dataWithMeta)
        
    ### if required filter on isGenotek
    if filterGenotek:
        print 'filtering out genotek samples'
        dataWithMeta=dataWithMeta[dataWithMeta['isGenotek']!=1]
        print 'row number after filtering is %s' %len(dataWithMeta)
        
    ### if required, filter on minimal number of reads
    if filterMinimalReads is not None:
        print 'filtering out samples with read number after HGF=%s' %filterMinimalReads
        dataWithMeta=dataWithMeta[dataWithMeta['PostHGF']>filterMinimalReads]
        print 'row number after filtering is %s' %len(dataWithMeta)
        
    ### indicating final number of BDs and FDs in the table:
    nBDs=dataWithMeta['BD'].nunique()
    nFDs=dataWithMeta['FD'].nunique()

    print 'final number of unique BDs is %s and unique FDs is %s' %(nBDs, nFDs)
    
    ### save to file:
    fileName='%s_SampleList%s_filterGenotek%s_filterMinimalRead%s' %(dataDFname,SampleListName,filterGenotek,
                                                                filterMinimalReads)
    if folderToSave is None:
        folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/FilteredDataTables'

    print 'saving filtered data table to folder %s' % folderToSave   
    if not isdir(folderToSave):
            makedirs(folderToSave)
    f1='%s/%s' %(folderToSave,fileName)    
    dataWithMeta.to_pickle(f1)
    
    return dataWithMeta
    
        
        
    

### RUN FUNCTION:

In [ ]:
# SPECIES LEVEL, PNP434samples, no additional filtering:

f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/notFiltered/MPA_s_Metabolon2_RA'
MPAlevel=pd.read_pickle(f1)

#LOAD pnp434 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP434samples','rb') as fp:
    PNP434samples=pickle.load(fp)
    

readCountFolder='Metabolon2'
dataDF=MPAlevel.reset_index()
dataDFname='MPA_s_Metabolon2'
dataMergeOn='FD'
BDFDMergeOn='FD'
SampleList=PNP434samples
SampleListName='PNP434'
filterGenotek=False
filterMinimalReads=None
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered'



dataWithMeta=filter_data_on_FDproperties(readCountFolder,dataDF,dataDFname,dataMergeOn,BDFDMergeOn,SampleList=SampleList,
                                SampleListName=SampleListName,filterGenotek=filterGenotek, filterMinimalReads=filterMinimalReads,
                               folderToSave=folderToSave)

In [ ]:
#species level, only pnp434 samples, only swabs, only readcounts>9000000


#LOAD DATA DF:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/notFiltered/MPA_s_Metabolon2_RA'
MPAlevel=pd.read_pickle(f1)

#LOAD pnp434 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP434samples','rb') as fp:
    PNP434samples=pickle.load(fp)

readCountFolder='Metabolon2'
dataDF=MPAlevel.reset_index()
dataDFname='MPA_s_Metabolon2'
dataMergeOn='FD'
BDFDMergeOn='FD'
SampleList=PNP434samples
SampleListName='PNP434'
filterGenotek=True
filterMinimalReads=9000000
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered'



dataWithMeta=filter_data_on_FDproperties(readCountFolder,dataDF,dataDFname,dataMergeOn,BDFDMergeOn,SampleList=SampleList,
                                SampleListName=SampleListName,filterGenotek=filterGenotek, filterMinimalReads=filterMinimalReads,
                               folderToSave=folderToSave)

## develop

In [ ]:
level='s'
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/notFiltered/MPA_%s_RA' %level
MPAlevel=pd.read_pickle(f1)

In [ ]:
MPAlevel.head()

In [ ]:
MPAlevel.shape

### merge with bd-fd table:

In [ ]:
#load bd-fd table (can change the BD-FD converter version by changing the date in the file name)
print 'loading BD_FD table'
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_16032018'
BD_FD=pd.read_pickle(f1)
print 'row number in BD_FD table is %s' %len(BD_FD)
BD_FD.head()


In [ ]:
#merging:
dataDF=MPAlevel.reset_index() #dataMergedOn shouldnt be the df index!
dataMergeOn='FD'
BDFDMergeOn='FD'


print 'merging dataDF with FD-BD table:'
dataWithMeta=pd.merge(dataDF,BD_FD,how='left', left_on=dataMergeOn, right_on=BDFDMergeOn)
print 'row number in dataDF before merging was %s' %len(dataDF)
print 'row number after merging is %s' %len(dataWithMeta)

dataWithMeta.head()

In [ ]:
dataWithMeta[dataWithMeta['PostHGF'].isnull()]

In [ ]:
dataDF[dataDF['PostHGF'].isnull()]

### if required - filter on specific BDs, if not-filter on samples that have any BD:

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP434samples','rb') as fp:
    pnp434samples=pickle.load(fp)
    
SampleList=pnp434samples
SampleListName='PNP434'
# SampleList=None


if SampleList is None:
    print 'filtering out all FD samples without BDs'
    dataWithMeta=dataWithMeta[dataWithMeta['BD'].notnull()]
    print 'row number after filtering is %s' %len(dataWithMeta)
else:
    print 'filtering out all FD that are not matched to any BD in the sample list' 
    dataWithMeta=dataWithMeta[dataWithMeta['BD'].isin(SampleList)]
    print 'row number after filtering is %s' %len(dataWithMeta)
    

### if required filter on isGenotek

In [ ]:
filterGenotek=True

if filterGenotek:
    print 'filtering out genotek samples'
    dataWithMeta=dataWithMeta[dataWithMeta['isGenotek']!=1]
    print 'row number after filtering is %s' %len(dataWithMeta)

### if required, filter on minimal number of reads

In [ ]:
filterMinimalReads=9000000

if filterMinimalReads is not None:
    print 'filtering out samples with read number after HGF=%s' %filterMinimalReads
    dataWithMeta=dataWithMeta[dataWithMeta['PostHGF']>filterMinimalReads]
    print 'row number after filtering is %s' %len(dataWithMeta)

### indicating final number of BDs and FDs in the table:

In [ ]:
nBDs=dataWithMeta['BD'].nunique()
nFDs=dataWithMeta['FD'].nunique()

print 'final number of unique BDs is %s and unique FDs is %s' %(nBDs, nFDs)

In [ ]:
266./434

### filtering out FDs with non complete match to BD - not implmented, as it seems that there no BDs with both complete and incomplete match...


In [ ]:
# filterIncompleteMatches=True

# if filterIncompleteMatches:
#     BDlist=list(dataWithMeta['BD'])
#     for BD in BDlist:
#         nBD=BDlist.count(BD)
# #         print BD,nBD
#         if nBD>1:
#             BDdf=dataWithMeta[dataWithMeta['BD']==BD]
#             FDlist=list(BDdf['FD'])
#             print BD,nBD,FDlist
            
#             nNonComments=len(BDdf[BDdf['Comment']!='Not the same Year!!'])
#             print nNonComments
# #             print BDdf['Comment']
#             if nNonComments>0:
#                 for n in BDdf.index:
#                     if BDdf.loc[n,'Comment']=='Not the same Year!!':
#                         print BDdf.loc[:,['BD','FD','Comment']]
#                         print 'dropping sample %s' %dataWithMeta.loc[n,'FD']
#                         print BDdf.loc[n,['BD','FD','Comment']]
#                         dataWithMeta=dataWithMeta.drop(n,axis=0) 
                        
            
            
            
                

### save filtered list


In [ ]:
folderToSave=None
dataDFname='MPA_s'


In [ ]:
fileName='%s_SampleList%s_filterGenotek%s_filterMinimalRead%s' %(dataDFname,SampleListName,filterGenotek,
                                                                filterMinimalReads)
if folderToSave is None:
    folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/FilteredDataTables'
    
print 'saving filtered data table to folder %s' % folderToSave
if not isdir(folderToSave):
            makedirs(folderToSave)

f1='%s/%s' %(folderToSave,fileName)    
dataWithMeta.to_pickle(f1)

In [ ]:
dataWithMeta.shape

In [ ]:
dataDF.shape

# merging on BD numbers

## function:

In [ ]:
'''
### THIS FUNCTION WAS COPIED TO Feature_phenotype_functions.py - UPDATE THERE!!!

the following functions takes DF tables that include 'BD' column and groups the data based on this column


input:
*DFtoGroup - sampleXphenotype dataframe, 'BD' column should exist
*DFtoGroupName - a string
*groupFunction='noOutlierMean'/'mean'/'mostFrequent'/'binarySelection'
*filterOutlierSamples=True/False
*filterSamePerson=True/False
*folderToSave=None/full path to folder

*nSTD - must be None if mergeFunction is not 'noOutlierMean' this defines the number of STD from the mean
to define low/high threshold for non-outlier samples. this apply to the BD merging procedure and not the 
outlier removal which is currently constant at 3. 
*nMinSamples - must be None if mergeFunction is not 'noOutlierMean'.this defines the minimal number of 
samples to apply samples filtering. this apply to the BD merging procedure and not the 
outlier removal which is currently constant at 3. 

'''

def mergeOnBD(DFtoGroup,DFtoGroupName,groupFunction,filterOutlierSamples,filterSamePerson,folderToSave,nSTD=None,
              nMinSamples=None):
    print 'loading again BD_FD table'
    f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_16032018'
    BD_FD=pd.read_pickle(f1)
    
    for column in DFtoGroup.columns.values:
        if column!='BD':
            if column in BD_FD.columns.values:
                DFtoGroup=DFtoGroup.drop(column,axis=1)
    
    
    ### group on BD, using the selected mergeFunction:
    print 'df length before grouping is %s' %len(DFtoGroup)
    print 'grouping on BD using %s' %groupFunction
    if groupFunction=='noOutlierMean':
        groupedDF=DFtoGroup.groupby('BD').agg(lambda x: noOutlierMean(x,nSTD,nMinSamples))
    elif groupFunction=='mean':
        groupedDF=DFtoGroup.groupby('BD').mean()
    elif groupFunction=='mostFrequent':   
        import collections
        groupedDF=DFtoGroup.groupby('BD').agg(lambda x: collections.Counter(x).most_common()[0][0])
    elif groupFunction=='binarySelection':
        groupedDF=DFtoGroup.groupby('BD').agg(lambda x: 1 if 1 in list(x) else 0)
    print 'grouped table length after grouping by BD and outlier removal=%s' %len(groupedDF)
    
    ### filter same person:
    if filterSamePerson:     
        # get again UserIDs for BD samples:

        BD_FD_toUse=BD_FD[['BD','UserID']]
        BD_FD_toUse=BD_FD_toUse.drop_duplicates()
        print 'number of BD-UserID pairs is %s' %len(BD_FD_toUse)

        print 'merging UserID info to BDgrouped table:'
        groupedDF=pd.merge(groupedDF,BD_FD_toUse,how='left',left_index=True,right_on='BD')

        # counting how many User IDs have more than one BD:
        BDrepeats=pd.DataFrame(groupedDF['UserID'].value_counts())
        nRepeatingBDs=len(BDrepeats[BDrepeats['UserID']>1])
        print '%s UserIDs have more than one BD' %nRepeatingBDs
        
        #dropping same person samples (leaving the last one)
        print 'groupedDF length before dropping same person samples is %s' %len(groupedDF)
        groupedDF=groupedDF.drop_duplicates(subset='UserID',keep='last')
        print 'groupedDF length after dropping same person samples is %s' %len(groupedDF)
        
    ### if required, filter outlier samples:
    if filterOutlierSamples:
        print 'filtering outlier samples...'
        groupedDF=filter_phenotypiclly_outlier_samples(groupedDF,nSTD=3,nMinSamples=3)
        print 'groupedDF length after dropping outlier samples is %s' %len(groupedDF)
        groupedDF=groupedDF.set_index('BD')
        
    ### save groupedDF:
    ### save to file:
    fileName='%s_%snSTD%snMinSamples%s_filterOutlierSample%s_filterSamePerson%s' %(DFtoGroupName,groupFunction,
                                                                nSTD,nMinSamples,filterOutlierSamples,
                                                                filterSamePerson,)

    if folderToSave is None:
        folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/MergedOnBDDataTables'

    print 'saving mergedOnBDs data table to folder %s' % folderToSave   
    if not isdir(folderToSave):
            makedirs(folderToSave)
    f1='%s/%s' %(folderToSave,fileName)    
    groupedDF.to_pickle(f1)
    
    print 'done!'
    
    return groupedDF    
    

    
    
    

### run function:

In [ ]:
##take pnp434 non filtered MPA_s and merge on BD,filter same person but not outliers:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone'
MPA_PNP434_notfiltered=pd.read_pickle(f1)


DFtoGroup=MPA_PNP434_notfiltered
DFtoGroupName='MPA_s_Metabolon2_PNP434_notfiltered'
groupFunction='noOutlierMean'
filterOutlierSamples=False
filterSamePerson=True
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD'
nSTD=3,
nMinSamples=3

groupedDF=mergeOnBD(DFtoGroup,DFtoGroupName,groupFunction,filterOutlierSamples,filterSamePerson,folderToSave,nSTD=nSTD,
              nMinSamples=nMinSamples)

In [ ]:
##take pnp434 non filtered MPA_s and merge on BD,filter same person but not outliers ### grouping function is mean!
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone'
MPA_PNP434_notfiltered=pd.read_pickle(f1)


DFtoGroup=MPA_PNP434_notfiltered
DFtoGroupName='MPA_s_Metabolon2_PNP434_notfiltered'
groupFunction='mean'
filterOutlierSamples=False
filterSamePerson=True
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD'
nSTD=3,
nMinSamples=3

groupedDF=mergeOnBD(DFtoGroup,DFtoGroupName,groupFunction,filterOutlierSamples,filterSamePerson,folderToSave,nSTD=nSTD,
              nMinSamples=nMinSamples)

In [ ]:
##take pnp434 MPA_s filtered on swabs and minimal read count and merge on BD,
#then filter same person but not outliers:

f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_Metabolon2_SampleListPNP434_filterGenotekTrue_filterMinimalRead9000000'
MPA_PNP434_filGenotek_fil9000000reads=pd.read_pickle(f1)


DFtoGroup=MPA_PNP434_filGenotek_fil9000000reads
DFtoGroupName='MPA_s_Metabolon2_PNP434_filGenotek_fil9000000reads'
groupFunction='noOutlierMean'
filterOutlierSamples=False
filterSamePerson=True
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD'
nSTD=3,
nMinSamples=3

groupedDF=mergeOnBD(DFtoGroup,DFtoGroupName,groupFunction,filterOutlierSamples,filterSamePerson,folderToSave,nSTD=nSTD,
              nMinSamples=nMinSamples)

In [ ]:
##take pnp434 MPA_s filtered on swabs and minimal read count and merge on BD,#### group function is mean!!
#then filter same person but not outliers:

f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_Metabolon2_SampleListPNP434_filterGenotekTrue_filterMinimalRead9000000'
MPA_PNP434_filGenotek_fil9000000reads=pd.read_pickle(f1)


DFtoGroup=MPA_PNP434_filGenotek_fil9000000reads
DFtoGroupName='MPA_s_Metabolon2_PNP434_filGenotek_fil9000000reads'
groupFunction='mean'
filterOutlierSamples=False
filterSamePerson=True
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD'
nSTD=3,
nMinSamples=3

groupedDF=mergeOnBD(DFtoGroup,DFtoGroupName,groupFunction,filterOutlierSamples,filterSamePerson,folderToSave,nSTD=nSTD,
              nMinSamples=nMinSamples)

## develop:

### Load filtered MPA table: 


In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_SampleListNone_filterGenotekFalse_filterMinimalReadNone'
MPAnotfiltered=pd.read_pickle(f1)
print len(MPAnotfiltered)

### group on BD, using the selected mergeFunction:

In [ ]:
DFtoGroup=MPAnotfiltered
groupFunction='noOutlierMean'
nSTD=3
nMinSamples=3

In [ ]:
print 'df length before grouping is %s' %len(DFtoGroup)
print 'grouping on BD using %s' %groupFunction
if groupFunction=='noOutlierMean':
    groupedDF=DFtoGroup.groupby('BD').agg(lambda x: noOutlierMean(x,nSTD,nMinSamples))
elif groupFunction=='mean':
    groupedDF=DFtoGroup.groupby('BD').mean()
elif groupFunction=='mostFrequent':   
    import collections
    groupedDF=DFtoGroup.groupby('BD').agg(lambda x: collections.Counter(x).most_common()[0][0])
elif groupFunction=='binarySelection':
    groupedDF=DFtoGroup.groupby('BD').agg(lambda x: 1 if 1 in list(x) else 0)
print 'grouped table length after grouping by BD and outlier removal=%s' %len(groupedDF)



In [ ]:
groupedDF.index.nunique()

In [ ]:
groupedDF.head()

### if required, filter outlier samples:

In [ ]:
filterOutlierSamples=True

In [ ]:
if filterOutlierSamples:
    print 'filtering outlier samples...'
    groupedDF=filter_phenotypiclly_outlier_samples(groupedDF,nSTD=3,nMinSamples=3)
    print 'groupedDF length after dropping outlier samples is %s' %len(groupedDF)

In [ ]:
groupedDF.head()

### filter same person:


In [ ]:
filterSamePerson=True

In [ ]:
if filterSamePerson:     
    # get again UserIDs for BD samples:
    print 'loading again BD_FD table'
    f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_16032018'
    BD_FD=pd.read_pickle(f1)
    BD_FD_toUse=BD_FD[['BD','UserID']]
    BD_FD_toUse=BD_FD_toUse.drop_duplicates()
    print 'number of BD-UserID pairs is %s' %len(BD_FD_toUse)

    print 'merging UserID info to BDgrouped table:'
    groupedDF=pd.merge(groupedDF,BD_FD_toUse,how='left',left_index=True,right_on='BD')


In [ ]:
# counting how many User IDs have more than one BD:
BDrepeats=pd.DataFrame(groupedDF['UserID'].value_counts())
nRepeatingBDs=len(BDrepeats[BDrepeats['UserID']>1])
print '%s UserIDs have more than one BD' %nRepeatingBDs

In [ ]:
print 'groupedDF length before dropping same person samples is %s' %len(groupedDF)
groupedDF=groupedDF.drop_duplicates(subset='UserID',keep='last')
print 'groupedDF length after dropping same person samples is %s' %len(groupedDF)
groupedDF=groupedDF.set_index('BD')


In [ ]:
groupedDF.head()

### save groupedDF:

In [ ]:
DFtoGroupName='MPA_s_notFiltered'
folderToSave=None

In [ ]:
### save to file:
fileName='%s_%snSTD%snMinSamples%s_filterOutlierSample%s_filterSamePerson%s' %(DFtoGroupName,groupFunction,
                                                            nSTD,nMinSamples,filterOutlierSamples,
                                                            filterSamePerson,)

if folderToSave is None:
    folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/MergedOnBDDataTables'

print 'saving mergedOnBDs data table to folder %s' % folderToSave   
if not isdir(folderToSave):
        makedirs(folderToSave)
f1='%s/%s' %(folderToSave,fileName)    
groupedDF.to_pickle(f1)

# check number of species in differnt files:

In [ ]:
# original mb file:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/notFiltered/MPA_s_Metabolon2_RA'
MPA_s_Metabolon2_RA=pd.read_pickle(f1)
print len(MPA_s_Metabolon2_RA.columns.values)
MPA_s_Metabolon2_RA.head()

In [ ]:
# filtered only to 434 samples:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone'
MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone=pd.read_pickle(f1)
print len(MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone.columns.values)
MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone.head()

In [ ]:
# filtered to 434 samples, swabs minimal read counts:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_Metabolon2_SampleListPNP434_filterGenotekTrue_filterMinimalRead9000000'
MPA_s_Metabolon2_SampleListPNP434_filterGenotekTrue_filterMinimalRead9000000=pd.read_pickle(f1)
print len(MPA_s_Metabolon2_SampleListPNP434_filterGenotekTrue_filterMinimalRead9000000.columns.values)
MPA_s_Metabolon2_SampleListPNP434_filterGenotekTrue_filterMinimalRead9000000.head()

In [ ]:
# filtered to 434 samples ONLY.merged onBD
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_Metabolon2_PNP434_notfiltered_noOutlierMeannSTD(3,)nMinSamples3_filterOutlierSampleFalse_filterSamePersonTrue'
MPA_s_notfiltered_mergedBD=pd.read_pickle(f1)
print len(MPA_s_notfiltered_mergedBD.columns.values)
MPA_s_notfiltered_mergedBD.head()

In [ ]:
# filtered to 434 samples ONLY.merged onBD
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_Metabolon2_PNP434_notfiltered_noOutlierMeannSTD(3,)nMinSamples3_filterOutlierSampleFalse_filterSamePersonTrue'
MPA_s_notfiltered_mergedBD=pd.read_pickle(f1)
print len(MPA_s_notfiltered_mergedBD.columns.values)
MPA_s_notfiltered_mergedBD.head()

In [ ]:
# filtered to 434 samples ONLY.merged onBD merge function=mean
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_Metabolon2_PNP434_notfiltered_meannSTD(3,)nMinSamples3_filterOutlierSampleFalse_filterSamePersonTrue'
MPA_s_notfiltered_mergedBD=pd.read_pickle(f1)
print len(MPA_s_notfiltered_mergedBD.columns.values)
MPA_s_notfiltered_mergedBD.head()

In [ ]:
for column in MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone.columns.values:
    if column not in MPA_s_notfiltered_mergedBD.columns.values:
        print column

In [ ]:
# filtered to 434 samples SWABS ONLY minimal reads.merged onBD
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_Metabolon2_PNP434_filGenotek_fil9000000reads_noOutlierMeannSTD(3,)nMinSamples3_filterOutlierSampleFalse_filterSamePersonTrue'
MPA_s_filtered_mergedBD=pd.read_pickle(f1)
print len(MPA_s_filtered_mergedBD.columns.values)
MPA_s_filtered_mergedBD.head()

In [ ]:
# filtered to 434 samples SWABS ONLY minimal reads.merged onBD merge function=mean
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_Metabolon2_PNP434_filGenotek_fil9000000reads_meannSTD(3,)nMinSamples3_filterOutlierSampleFalse_filterSamePersonTrue'
MPA_s_filtered_mergedBD=pd.read_pickle(f1)
print len(MPA_s_filtered_mergedBD.columns.values)
MPA_s_filtered_mergedBD.head()

In [ ]:
n=0
for column in MPA_s_filtered_mergedBD.columns.values:
    if column not in MPA_s_notfiltered_mergedBD.columns.values:
        print column
        n=n+1
print n

In [ ]:
n=0
for column in MPA_s_notfiltered_mergedBD.columns.values:
    if column not in MPA_s_filtered_mergedBD.columns.values:
        print column
        n=n+1
        
print n

In [ ]:
df1=MPA_s_Metabolon2_SampleListPNP434_filterGenotekFalse_filterMinimalReadNone
#check for nans:
df[df['Acidaminococcus_fermentans'].isnull()]

In [ ]:
df['Acidaminococcus_fermentans'].sum()

In [ ]:
df['Acidaminococcus_fermentans'].count()

In [ ]:
a=1
a>=2

In [ ]:
a>=0

In [ ]:
a>=1

In [ ]:
a>1